In [1]:
import itertools
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from multiprocessing import Pool, cpu_count

import sys
sys.path.append(r"D:\ProyectosGit\IA\Tarea_Final_IA")

from src.controlador_duelos import ControladorDuelos
from src.selectores_de_oponentes import SelectorAllForOne
from src.strategies import SARSA, QLearning, TitForTat, Davis, Downing, Feld, Grofman, SiempreCoopera, SiempreTraiciona
from src.strategies.RL.Estados import StatState, HistoryState, HistoryStatState
from src.strategies.RL.politicas import EpsilonGreedy

In [4]:
from src.rl_tuning.tuning_functions import run_crecimiento_en_paralelo, generate_configs, configs_politicas, run_crecimiento_en_paralelo_politicas


In [3]:
configs = generate_configs()

df_crecimiento = run_crecimiento_en_paralelo(configs, [4,5,6])

Ejecutando 702 experimentos en paralelo...
Usando 6 procesos


100%|██████████████████████████████████████████████████████████████████████████████| 702/702 [2:12:46<00:00, 11.35s/it]


In [19]:
df_crecimiento2 = df_crecimiento.copy()
df_crecimiento2.sort_values(by="id").tail()

,historial,memory,grupos,state_manager,algo,alpha,id,mean_eval 0,exploracion 0,mean_eval 1,...,mean_eval 5,exploracion 5,mean_eval 6,exploracion 6,mean_eval 7,exploracion 7,mean_eval 8,exploracion 8,mean_eval 9,exploracion 9
696,2.0,15.0,5.0,mix,SARSA,0.1,233,1083.4,0.119556,1206.8,...,909.0,0.152889,846.5,0.153778,1088.8,0.155111,914.0,0.159556,529.0,0.168889
698,2.0,15.0,5.0,mix,SARSA,0.1,233,741.2,0.119556,902.0,...,1037.5,0.158667,581.0,0.160000,946.2,0.164000,1003.9,0.164444,665.8,0.164889
700,3.0,15.0,5.0,mix,SARSA,0.1,234,581.0,0.043592,963.9,...,1004.0,0.058367,739.8,0.059347,538.0,0.061469,746.0,0.062694,1194.5,0.063347
699,3.0,15.0,5.0,mix,SARSA,0.1,234,527.0,0.048408,1107.8,...,1038.9,0.058531,579.0,0.061388,997.7,0.061796,1071.0,0.062939,955.0,0.063673
701,3.0,15.0,5.0,mix,SARSA,0.1,234,977.0,0.040490,911.3,...,902.5,0.056816,975.1,0.057633,554.0,0.059184,559.0,0.059837,558.2,0.060245


In [21]:
valores = df_crecimiento2[df_crecimiento2["id"] == 187]["mean_eval 0"].tolist()
valores = valores + valores
valores = np.array(valores)
print(valores)

for id in range (235):
    
    for i in range(10):
        

[ 900.1 1088.7 1000.2  900.1 1088.7 1000.2]
0
1


In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

resultados = []

for id_actual in range(0, 235):

    # Filtrar las 3 filas de esa ID
    df_id = df_crecimiento2[df_crecimiento2["id"] == id_actual]

    if df_id.empty:
        continue

    # Columnas a usar
    columnas_valores = [f"mean_eval {i}" for i in range(10)]

    # Extraer matriz 3x10 → luego a vector 30x1
    valores = df_id[columnas_valores].values.flatten()   # 30 valores

    # Crear X: 0..9 repetidos 3 veces → 30 filas
    X = np.tile(np.arange(10), 3).reshape(-1, 1)

    y = valores

    # Entrenar regresión lineal
    modelo = LinearRegression()
    modelo.fit(X, y)

    # Predicción
    y_pred = modelo.predict(X)

    # Pendiente (crecimiento)
    pendiente = modelo.coef_[0]

    # Error MAE
    mae = mean_absolute_error(y, y_pred)

    # Guardar resultados
    resultados.append({
        "id": id_actual,
        "crecimiento": pendiente,
        "mae": mae
    })

# DataFrame final
df_resultados = pd.DataFrame(resultados)

df_resultados


,id,crecimiento,mae
0,1,-13.988687,54.550249
1,2,-6.331313,142.643003
2,3,5.122020,163.306397
3,4,-3.048485,54.951919
4,5,6.117374,142.409993
...,...,...,...
229,230,-0.959192,113.300741
230,231,14.690101,165.363960
231,232,0.360808,118.598168
232,233,-11.837980,104.083852


In [46]:
df_resultados.sort_values(by="crecimiento").tail(40)

,id,crecimiento,mae
219,220,7.763636,126.208242
114,115,8.398586,142.679650
55,56,8.447273,83.347030
69,70,8.814141,63.219933
11,12,8.863636,128.734545
149,150,9.063030,156.545010
75,76,9.507677,101.077684
102,103,9.552727,106.674303
10,11,9.701010,126.769589
87,88,9.916162,131.092027


In [53]:
df_crecimiento2[df_crecimiento2["id"] == 188]

,historial,memory,grupos,state_manager,algo,alpha,id,mean_eval 0,exploracion 0,mean_eval 1,...,mean_eval 5,exploracion 5,mean_eval 6,exploracion 6,mean_eval 7,exploracion 7,mean_eval 8,exploracion 8,mean_eval 9,exploracion 9
561,2.0,15.0,3.0,mix,Q,0.1,188,677.7,0.242798,1072.9,...,997.1,0.292181,915.0,0.298354,881.0,0.298354,1257.5,0.298354,1081.4,0.300412
562,2.0,15.0,3.0,mix,Q,0.1,188,1012.4,0.222222,765.0,...,1056.3,0.273663,1180.9,0.275720,936.0,0.294239,930.0,0.294239,997.3,0.294239
563,2.0,15.0,3.0,mix,Q,0.1,188,512.0,0.220165,562.0,...,921.0,0.302469,916.0,0.304527,1073.7,0.304527,1009.1,0.304527,1142.8,0.306584


In [5]:
configs_politica = configs_politicas()

df_politicas = run_crecimiento_en_paralelo_politicas(configs_politica, [7,8,9,10],episodes_train = 400)

Ejecutando 48 experimentos en paralelo...
Usando 6 procesos


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [31:34<00:00, 39.47s/it]


In [6]:
df_politicas

,tau,ep_base,ep_fin,politica,id,mean_eval 0,exploracion 0,mean_eval 1,exploracion 1,mean_eval 2,...,mean_eval 5,exploracion 5,mean_eval 6,exploracion 6,mean_eval 7,exploracion 7,mean_eval 8,exploracion 8,mean_eval 9,exploracion 9
0,1.0,NaN,NaN,softmax,1,915.0,0.146091,915.0,0.146091,915.0,...,915.0,0.146091,915.0,0.146091,915.0,0.146091,915.0,0.146091,915.0,0.146091
1,2.0,NaN,NaN,softmax,2,911.0,0.162551,911.0,0.162551,911.0,...,911.0,0.162551,911.0,0.162551,911.0,0.162551,911.0,0.162551,911.0,0.162551
2,1.0,NaN,NaN,softmax,1,913.0,0.106996,913.0,0.106996,913.0,...,913.0,0.106996,913.0,0.106996,913.0,0.106996,913.0,0.106996,913.0,0.106996
3,1.0,NaN,NaN,softmax,1,820.0,0.176955,820.0,0.176955,820.0,...,820.0,0.176955,820.0,0.176955,820.0,0.176955,820.0,0.176955,820.0,0.176955
4,1.0,NaN,NaN,softmax,1,908.0,0.156379,908.0,0.156379,908.0,...,908.0,0.156379,908.0,0.156379,908.0,0.156379,908.0,0.156379,908.0,0.156379
5,2.0,NaN,NaN,softmax,2,579.0,0.164609,579.0,0.164609,579.0,...,579.0,0.164609,579.0,0.164609,579.0,0.164609,579.0,0.164609,579.0,0.164609
6,3.0,NaN,NaN,softmax,3,883.0,0.162551,883.0,0.162551,883.0,...,883.0,0.162551,883.0,0.162551,883.0,0.162551,883.0,0.162551,883.0,0.162551
7,2.0,NaN,NaN,softmax,2,919.0,0.162551,919.0,0.162551,919.0,...,919.0,0.162551,919.0,0.162551,919.0,0.162551,919.0,0.162551,919.0,0.162551
8,2.0,NaN,NaN,softmax,2,937.0,0.187243,937.0,0.197531,937.0,...,937.0,0.209877,937.0,0.209877,937.0,0.209877,937.0,0.209877,937.0,0.209877
9,3.0,NaN,NaN,softmax,3,915.0,0.226337,1029.9,0.226337,915.0,...,887.0,0.232510,915.0,0.232510,895.6,0.232510,915.0,0.232510,915.0,0.232510


In [3]:
configs = generate_configs()

df = run_experiments(configs, [1,2,3])

configs: 100%|███████████████████████████████████████████████████████████████████████| 72/72 [1:18:00<00:00, 65.00s/it]


In [5]:
df.to_csv("Tuneo.csv")

In [11]:
df_copy = df.copy()
df_copy

,historial,memory,grupos,state_manager,algo,alpha,seed,train_mean,train_std,eval_mean,eval_std,eval_median,eval_p25,eval_p75,p_explorado
0,3.0,NaN,NaN,history,Q,0.01,1,803.2180,164.068475,929.00,0.000000,929.0,929.0,929.0,0.714286
1,3.0,NaN,NaN,history,Q,0.01,2,786.3495,157.368746,914.00,0.000000,914.0,914.0,914.0,0.734694
2,3.0,NaN,NaN,history,Q,0.01,3,797.0880,162.773712,920.00,0.000000,920.0,920.0,920.0,0.724490
3,3.0,NaN,NaN,history,Q,0.05,1,826.6775,170.177770,1084.14,8.414297,1084.0,1078.0,1091.5,0.704082
4,3.0,NaN,NaN,history,Q,0.05,2,825.8395,172.319572,858.00,0.000000,858.0,858.0,858.0,0.734694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,NaN,15.0,5.0,stats_summary,SARSA,0.05,2,805.5040,165.610178,916.00,0.000000,916.0,916.0,916.0,0.560000
212,NaN,15.0,5.0,stats_summary,SARSA,0.05,3,809.0200,168.277713,916.00,0.000000,916.0,916.0,916.0,0.536000
213,NaN,15.0,5.0,stats_summary,SARSA,0.10,1,825.1100,172.498794,916.00,0.000000,916.0,916.0,916.0,0.544000
214,NaN,15.0,5.0,stats_summary,SARSA,0.10,2,824.3145,173.662444,916.00,0.000000,916.0,916.0,916.0,0.552000


In [8]:
df_copy = summarize_results(df_copy)